In [1]:
!pip install matplotlib
!pip install seaborn
!pip install pandas
!pip install numpy
!pip install pyspark
!pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

movie_path = '../data/movies_data.csv'
cast_path = '../data/actor_actress_director.csv'
genre_path = '../data/genres_data.csv'
keyword_path = '../data/keyword_data.csv'
production_companies_path = '../data/production_companies_data.csv'

movies_df = pd.read_csv(
    movie_path,
    low_memory=False
)

actor_actress_director_df = pd.read_csv(
    cast_path,
    low_memory=False
)

genres_df = pd.read_csv(
    genre_path,
    low_memory=False
)

keywords_df = pd.read_csv(
    keyword_path,
    low_memory=False
)

prod_comp_df = pd.read_csv(
    production_companies_path,
    low_memory=False
)

print(movies_df.head())
print(movies_df.dtypes)

print(actor_actress_director_df.head())
print(actor_actress_director_df.dtypes)

print(genres_df.head())
print(genres_df.dtypes)

print(keywords_df.head())
print(keywords_df.dtypes)

print(prod_comp_df.head())
print(prod_comp_df.dtypes)

      tconst titleType                   primaryTitle  isAdult  releaseYear  \
0  tt0000009     movie                     Miss Jerry        0         1894   
1  tt0000147     movie  The Corbett-Fitzsimmons Fight        0         1897   
2  tt0000574     movie    The Story of the Kelly Gang        0         1906   
3  tt0000591     movie               The Prodigal Son        0         1907   
4  tt0000615     movie             Robbery Under Arms        0         1907   

   releaseMonth releaseDate  runtime_consolidated  imdb_averageRating  \
0            10  1894-10-08                  45.0                 5.4   
1             1  1897-01-01                 100.0                 5.3   
2            12  1906-12-26                  70.0                 6.0   
3             6  1907-06-19                  90.0                 5.6   
4            10  1907-10-01                  60.0                 4.3   

   imdb_numVotes  ...  bayesian_averageRating  profit_ratio  \
0          218.0  ...  

In [3]:
# for movies_df --> Genres, Keywords, and Proudction Companies strings --> split by commas, yeet duplicates, join back as string
movies_df['genres'] = movies_df['genres'].apply(lambda x: ','.join(sorted(set(x.split(',')))) if isinstance(x, str) else x)
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: ','.join(sorted(set(x.split(',')))) if isinstance(x, str) else x)
movies_df['production_companies'] = movies_df['production_companies'].apply(lambda x: ','.join(sorted(set(x.split(',')))) if isinstance(x, str) else x)

In [4]:
# flatten actors, actresses, directors --> new df and assign to individual columns as comma delimited string
actors_df = actor_actress_director_df[actor_actress_director_df['category'] == 'actor'] \
    .groupby('tconst')['primaryName'].apply(lambda x: ','.join(sorted(set(x)))).reset_index(name='actors')

actresses_df = actor_actress_director_df[actor_actress_director_df['category'] == 'actress'] \
    .groupby('tconst')['primaryName'].apply(lambda x: ','.join(sorted(set(x)))).reset_index(name='actresses')

directors_df = actor_actress_director_df[actor_actress_director_df['category'] == 'director'] \
    .groupby('tconst')['primaryName'].apply(lambda x: ','.join(sorted(set(x)))).reset_index(name='directors')

movies_df = movies_df.merge(actors_df, on='tconst', how='left')
movies_df = movies_df.merge(actresses_df, on='tconst', how='left')
movies_df = movies_df.merge(directors_df, on='tconst', how='left')

In [5]:
print(movies_df.head())
print(movies_df.dtypes)
print(movies_df.isnull().sum())

      tconst titleType                   primaryTitle  isAdult  releaseYear  \
0  tt0000009     movie                     Miss Jerry        0         1894   
1  tt0000147     movie  The Corbett-Fitzsimmons Fight        0         1897   
2  tt0000574     movie    The Story of the Kelly Gang        0         1906   
3  tt0000591     movie               The Prodigal Son        0         1907   
4  tt0000615     movie             Robbery Under Arms        0         1907   

   releaseMonth releaseDate  runtime_consolidated  imdb_averageRating  \
0            10  1894-10-08                  45.0                 5.4   
1             1  1897-01-01                 100.0                 5.3   
2            12  1906-12-26                  70.0                 6.0   
3             6  1907-06-19                  90.0                 5.6   
4            10  1907-10-01                  60.0                 4.3   

   imdb_numVotes  ...  log_votes  log_popularity  \
0          218.0  ...   5.402677  

In [6]:
# deal w/ missing values
cols_with_nulls = ['genres', 'production_companies', 'keywords', 'actors', 'actresses', 'directors']
movies_df[cols_with_nulls] = movies_df[cols_with_nulls].fillna('Unknown')

print(movies_df.head())
print(movies_df.isnull().sum())

      tconst titleType                   primaryTitle  isAdult  releaseYear  \
0  tt0000009     movie                     Miss Jerry        0         1894   
1  tt0000147     movie  The Corbett-Fitzsimmons Fight        0         1897   
2  tt0000574     movie    The Story of the Kelly Gang        0         1906   
3  tt0000591     movie               The Prodigal Son        0         1907   
4  tt0000615     movie             Robbery Under Arms        0         1907   

   releaseMonth releaseDate  runtime_consolidated  imdb_averageRating  \
0            10  1894-10-08                  45.0                 5.4   
1             1  1897-01-01                 100.0                 5.3   
2            12  1906-12-26                  70.0                 6.0   
3             6  1907-06-19                  90.0                 5.6   
4            10  1907-10-01                  60.0                 4.3   

   imdb_numVotes  ...  log_votes  log_popularity  \
0          218.0  ...   5.402677  

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_features = ['actors', 'actresses', 'directors', 'genres', 'keywords', 'production_companies']
numerical_features = ['runtime_consolidated', 'releaseMonth']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

In [13]:
# model timed not, not enough time to wait for it to finish
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

Q1 = movies_df['aaa_index'].quantile(0.25)
Q3 = movies_df['aaa_index'].quantile(0.75)
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
benchmark = upper_bound

movies_df['hit_movie'] = (movies_df['aaa_index'] > benchmark).astype(int)

categorical_features = ['actors', 'actresses', 'directors', 'genres', 'keywords', 'production_companies']
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_features)],
    remainder='passthrough'
)

numerical_features = ['runtime_consolidated', 'releaseMonth']
numerical_transformer = StandardScaler()

feature_set = [
    'actors', 
    'actresses',
    'directors',
    'genres',
    'keywords',
    'production_companies',
    'runtime_consolidated',
    'releaseMonth'
]

X = movies_df[feature_set]
y = movies_df['hit_movie']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

param_grid_rf = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [10, 20, None],
    'classifier__min_samples_split': [2, 5],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__class_weight': ['balanced', 'balanced_subsample']  # To handle class imbalance
}

grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_rf = grid_search_rf.best_estimator_

y_pred_rf = best_rf.predict(X_test)

print(f"Best Random Forest Parameters: {grid_search_rf.best_params_}")
print(classification_report(y_test, y_pred_rf))


KeyboardInterrupt: 

In [ ]:
# only here to test, initial model used timed out 
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42))
])

param_grid_xgb = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__max_depth': [3, 5, 7],
    'classifier__min_child_weight': [1, 3, 5],
    'classifier__subsample': [0.8, 1.0],
    'classifier__colsample_bytree': [0.8, 1.0],
    'classifier__scale_pos_weight': [1, 10, 50], 
    'classifier__eval_metric': ['logloss', 'error']
}

grid_search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)

best_xgb = grid_search_xgb.best_estimator_

y_pred_xgb = best_xgb.predict(X_test)

print(f"Best XGBoost Parameters: {grid_search_xgb.best_params_}")
print(classification_report(y_test, y_pred_xgb))